In [4]:
import psycopg
from dotenv import load_dotenv
import os
load_dotenv()

DB_NAME = os.getenv("DB_NAME", "lexai_db")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASSWORD = os.getenv("DB_PASSWORD","") 
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")

try:
    conn = psycopg.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,  # use your real password
        host=DB_HOST,
        port=DB_PORT,
    )
    print("✅ Connection successful!")
except Exception as e:
    print("❌ Connection failed:", e)

✅ Connection successful!


In [5]:
with conn.cursor() as cur:
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

In [6]:
create_decisions_sql = """
CREATE TABLE IF NOT EXISTS decisions (
    id          BIGSERIAL PRIMARY KEY,
    case_no     TEXT NOT NULL UNIQUE,    -- "G.R. No. 161796"
    division    TEXT,                    -- "THIRD DIVISION"
    title       TEXT,                    -- may be real title later; for now what you have
    created_at  TIMESTAMPTZ DEFAULT NOW(),
    updated_at  TIMESTAMPTZ DEFAULT NOW()
);
"""

with conn.cursor() as cur:
    cur.execute(create_decisions_sql)

In [7]:
create_chunks_sql = """
CREATE TABLE IF NOT EXISTS decision_chunks (
    id              BIGSERIAL PRIMARY KEY,

    -- FK via decision_id for efficient joins
    decision_id     BIGINT NOT NULL REFERENCES decisions(id) ON DELETE CASCADE,

    -- your metadata
    case_no         TEXT NOT NULL,     -- denormalized for convenience/debug
    section         TEXT,              -- 'PREAMBLE', 'FACTS', 'RULING', etc.
    chunk_index     INT NOT NULL,      -- 0,1,2,... within that case
    text            TEXT NOT NULL,

    -- for later / RAG
    token_count     INT,               -- optional (fill when you tokenize)
    embedding       VECTOR(1024),      -- BGE-M3 embedding

    created_at      TIMESTAMPTZ DEFAULT NOW(),
    updated_at      TIMESTAMPTZ DEFAULT NOW()
);
"""

with conn.cursor() as cur:
    cur.execute(create_chunks_sql)

In [8]:
create_indexes_sql = """
-- Vector ANN index (tune lists as your corpus grows)
CREATE INDEX IF NOT EXISTS idx_decision_chunks_embedding
ON decision_chunks
USING ivfflat (embedding vector_cosine_ops)
WITH (lists = 100);

-- Navigate within a decision
CREATE INDEX IF NOT EXISTS idx_decision_chunks_decision_idx
ON decision_chunks (decision_id, chunk_index);

CREATE INDEX IF NOT EXISTS idx_decision_chunks_case_no
ON decision_chunks (case_no);

CREATE INDEX IF NOT EXISTS idx_decisions_case_no
ON decisions (case_no);
"""

with conn.cursor() as cur:
    cur.execute(create_indexes_sql)


In [10]:
trigger_fn_sql = """
CREATE OR REPLACE FUNCTION set_updated_at()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
"""

with conn.cursor() as cur:
    cur.execute(trigger_fn_sql)

triggers_sql = """
DO $$
BEGIN
    IF NOT EXISTS (
        SELECT 1 FROM pg_trigger WHERE tgname = 'trg_decisions_set_updated_at'
    ) THEN
        CREATE TRIGGER trg_decisions_set_updated_at
        BEFORE UPDATE ON decisions
        FOR EACH ROW
        EXECUTE FUNCTION set_updated_at();
    END IF;

    IF NOT EXISTS (
        SELECT 1 FROM pg_trigger WHERE tgname = 'trg_decision_chunks_set_updated_at'
    ) THEN
        CREATE TRIGGER trg_decision_chunks_set_updated_at
        BEFORE UPDATE ON decision_chunks
        FOR EACH ROW
        EXECUTE FUNCTION set_updated_at();
    END IF;
END $$;
"""

with conn.cursor() as cur:
    cur.execute(triggers_sql)


In [ ]:
from chunking import build_rag_chunks
import json

data = []
with open('../data/sc_elibrary_decisions_text_combined_cleaned.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

full_text = data[0]['text']

chunks = build_rag_chunks(
    full_text,
    max_tokens=350,
    overlap_sentences=2
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
import sys
sys.path.append("/root/dev/PhilLexRAG")  # <- your project root (where config.py lives)

from upsert import insert_chunk_safe
from db_init import create_connection, close_connection

conn = create_connection()

✅ Connection successful!


In [2]:
from chunking import build_rag_chunks
import json

data = []
with open('../data/sc_elibrary_decisions_text_combined_cleaned.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

full_text = data[0]['text']

chunks = build_rag_chunks(
    full_text,
    max_tokens=350,
    overlap_sentences=2
)

conn = create_connection()
for ch in chunks:
    cid = insert_chunk_safe(conn, ch)
    print("Inserted chunk id:", cid)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ Connection successful!
Inserted chunk id: 1
Inserted chunk id: 2
Inserted chunk id: 3
Inserted chunk id: 4
Inserted chunk id: 5
Inserted chunk id: 6
Inserted chunk id: 7
Inserted chunk id: 8
Inserted chunk id: 9
Inserted chunk id: 10
Inserted chunk id: 11
Inserted chunk id: 12
Inserted chunk id: 13
Inserted chunk id: 14
Inserted chunk id: 15
Inserted chunk id: 16
Inserted chunk id: 17
Inserted chunk id: 18
Inserted chunk id: 19
Inserted chunk id: 20
Inserted chunk id: 21
Inserted chunk id: 22
Inserted chunk id: 23
Inserted chunk id: 24
Inserted chunk id: 25
Inserted chunk id: 26
Inserted chunk id: 27
Inserted chunk id: 28
Inserted chunk id: 29
Inserted chunk id: 30
Inserted chunk id: 31
Inserted chunk id: 32
Inserted chunk id: 33
Inserted chunk id: 34
Inserted chunk id: 35
Inserted chunk id: 36
Inserted chunk id: 37
Inserted chunk id: 38
Inserted chunk id: 39
Inserted chunk id: 40


In [3]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM decision_chunks;")
    rows = cur.fetchall()

print("Chunks with embeddings:", rows)

Chunks with embeddings: [(1, 1, 'G.R. No. 161796', 'PREAMBLE', 0, 'THIRD DIVISION\n[ G.R. No. 161796, February 08, 2012 ]\nLAND BANK OF THE PHILIPPINES, PETITIONER, VS. ESTATE OF J. AMADO ARANETA, RESPONDENT. [G.R. NO. 161830]\nDEPARTMENT OF AGRARIAN REFORM,\n[1]\nPETITIONER, NORBERTO RESULTA, EDITHA ABAD, LEDELIA ASIDOY, GIL PAGARAGAN, ROSALITO PAGHUBASAN, EDWIN FAUSTINO, FELOMINO JUSOL, EDELBERTO POBLARES, EFREN APON, NELSON VILLAREAL, JIMMY ZONIO, SERLISTO ZONIO, WILFREDO MARCELINO, ROGELIO RODERO, SERGIO ZONIO, NORBERTO FRANCISCO, AURORA VILLACORTE, JOVITO NINONUEVO, ELIZABETH ZAUSA, RUBEN VILLANUEVA, VICENTA RACCA, ROGELIO RACCA, MERCEDES VILLANUEVA, EDUARDO BIUTE, APOLINARIO TORRAL, BENJAMIN TANJER, JR., MINDA SOLIMAN, CIPRIANO REQUIOLA, GLORIA ROMERO, SILVERIO ZONIO, NESTOR ZONIO, NILO ZAUSA, ROMUALDO ZAUSA, REYNALDO ZAUSA, LUMILYN ZAUSA, GILBERT BAUTISTA, GILDA PACETES, ALUDIA CALUB, LOURDES CAGNO, ABELARDO CAGNO, BENJAMIN MARINAS, CRISPINA ARNAIZ, MARIA CABUS, RESTITUTA PRETEN

In [4]:
close_connection(conn)

Connection closed.
